In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY="5g"
spark = SparkSession.builder.appName("taxi_fare_pred")\
        .config("spark.executor.memory", MAX_MEMORY)\
        .config("spark.driver.memory", MAX_MEMORY)\
        .getOrCreate()

In [3]:
trip_files = "/Users/ji/data-engineering/01-spark/data/trips/*"

In [4]:
trips_df = spark.read.parquet(f"file:///{trip_files}", inferSchema=True, header=True)

In [5]:
trips_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [6]:
trips_df.createOrReplaceTempView("trips")

In [7]:
query = """
SELECT
    passenger_count,
    PULocationID as pickup_location_id,
    DOLocationID as dropoff_location_id,
    trip_distance,
    HOUR(tpep_pickup_datetime) as pickup_time,
    DATE_FORMAT(TO_DATE(tpep_pickup_datetime), 'EEEE') as day_of_week,
    total_amount
FROM
    trips
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-08-01'
"""

data_df = spark.sql(query)
data_df.createOrReplaceTempView('data')

In [8]:
data_df.show()

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|            1.0|                79|                116|          8.4|          9|   Thursday|       35.15|
|            1.0|                75|                236|          0.9|          9|   Thursday|         8.8|
|            1.0|               236|                168|          3.4|          9|   Thursday|        15.3|
|            1.0|               238|                152|         1.96|          9|   Thursday|       13.39|
|            1.0|               142|                238|         0.77|          9|   Thursday|        9.54|
|            1.0|               238|                244|         3.65|          9|   Thursday|       15.36|
|            0.0|           

In [9]:
data_df.printSchema()

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: long (nullable = true)
 |-- dropoff_location_id: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [10]:
train_df, test_df = data_df.randomSplit([0.8,0.2], seed=1)

In [11]:
data_dir = "/Users/ji/data-engineering/01-spark/data/"

In [12]:
train_df.write.format('parquet').save(f"{data_dir}/train/")
test_df.write.format('parquet').save(f"{data_dir}/test/")

In [13]:
train_df = spark.read.parquet(f"{data_dir}/train/")
test_df = spark.read.parquet(f"{data_dir}/test/")

In [14]:
train_df.printSchema()

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: long (nullable = true)
 |-- dropoff_location_id: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [17]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

cat_feats = [
    "pickup_location_id",
    "dropoff_location_id",
    "day_of_week"
]

stages = []

for c in cat_feats:
    cat_indexer = StringIndexer(inputCol=c, outputCol=c + "_idx").setHandleInvalid('keep')
    onehot_encoder = OneHotEncoder(inputCols=[cat_indexer.getOutputCol()], outputCols=[c+"_onehot"])
    stages += [cat_indexer, onehot_encoder]

In [19]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

num_feats = [
    "passenger_count",
    "trip_distance",
    "pickup_time"
]

for n in num_feats:
    num_assembler = VectorAssembler(inputCols=[n], outputCol = n+"_vector")
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol=n+"_scaled")
    stages += [num_assembler, num_scaler]

In [26]:
assembler_inputs = [c+"_onehot" for c in cat_feats] + [n+"_scaled" for n in num_feats]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol='feature_vector')
stages += [assembler]

In [27]:
from pyspark.ml import Pipeline

transform_stages = stages
pipeline = Pipeline(stages=transform_stages)
fitted_transformer = pipeline.fit(train_df)

In [28]:
vtrain_df = fitted_transformer.transform(train_df)

In [29]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    maxIter=5,
    solver = "normal",
    labelCol="total_amount",
    featuresCol = "feature_vector"
)

In [30]:
vtrain_df.printSchema()

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: long (nullable = true)
 |-- dropoff_location_id: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id_idx: double (nullable = false)
 |-- pickup_location_id_onehot: vector (nullable = true)
 |-- dropoff_location_id_idx: double (nullable = false)
 |-- dropoff_location_id_onehot: vector (nullable = true)
 |-- day_of_week_idx: double (nullable = false)
 |-- day_of_week_onehot: vector (nullable = true)
 |-- passenger_count_vector: vector (nullable = true)
 |-- passenger_count_scaled: vector (nullable = true)
 |-- trip_distance_vector: vector (nullable = true)
 |-- trip_distance_scaled: vector (nullable = true)
 |-- pickup_time_vector: vector (nullable = true)
 |-- pickup_time_scaled: vector (nullable = true)
 |-- feature_vector: vector (nullable =

In [31]:
model = lr.fit(vtrain_df)

22/06/14 00:14:08 WARN Instrumentation: [1d632ceb] regParam is zero, which might cause numerical instability and overfitting.
22/06/14 00:14:17 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/06/14 00:14:17 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/06/14 00:14:24 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
22/06/14 00:14:25 WARN Instrumentation: [1d632ceb] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
22/06/14 00:14:25 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/06/14 00:14:25 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [32]:
vtest_df = fitted_transformer.transform(test_df)

In [34]:
pred = model.transform(vtest_df)

In [35]:
pred.cache()

DataFrame[passenger_count: double, pickup_location_id: bigint, dropoff_location_id: bigint, trip_distance: double, pickup_time: int, day_of_week: string, total_amount: double, pickup_location_id_idx: double, pickup_location_id_onehot: vector, dropoff_location_id_idx: double, dropoff_location_id_onehot: vector, day_of_week_idx: double, day_of_week_onehot: vector, passenger_count_vector: vector, passenger_count_scaled: vector, trip_distance_vector: vector, trip_distance_scaled: vector, pickup_time_vector: vector, pickup_time_scaled: vector, feature_vector: vector, prediction: double]

In [39]:
pred.select(['trip_distance', 'day_of_week', 'total_amount', 'prediction']).show()

+-------------+-----------+------------+------------------+
|trip_distance|day_of_week|total_amount|        prediction|
+-------------+-----------+------------+------------------+
|         13.9|   Saturday|         4.3|124.46664866216771|
|          3.3|     Sunday|         5.8|18.475916951038055|
|          5.5|     Sunday|       26.75|23.181156562027528|
|          7.9|   Saturday|        30.3|29.473850697733745|
|          0.4|  Wednesday|         7.8|12.294158587082581|
|          1.0|   Saturday|        11.8|12.966433437712235|
|          1.3|   Saturday|       12.85|14.015173645703879|
|          9.4|   Saturday|        39.8| 35.13642636791676|
|          1.5|  Wednesday|        13.3|15.534087785564875|
|          4.7|     Sunday|       26.15| 21.65279902002861|
|          0.6|   Saturday|       11.75| 13.40733753713533|
|          1.1|    Tuesday|        11.8|14.563621902757673|
|          3.7|     Friday|       24.35| 19.69203193420634|
|          2.4|   Saturday|       14.75|

In [41]:
model.summary.rootMeanSquaredError

5.675820016291852

In [42]:
model.summary.r2

0.806108106129646